In [98]:
#!pip install autogluon
#!pip install pandas

In [99]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection._split import _BaseKFold
import numpy as np
import random

In [100]:
seed_value = 42  # Replace with your desired seed value
random.seed(seed_value)
np.random.seed(seed_value)

In [101]:
x_train_c = pd.read_csv('cleaned_data_quarterly/C/x_train_c.csv')
y_train_c = pd.read_csv('cleaned_data_quarterly/C/train_c.csv')
x_test_c = pd.read_csv('cleaned_data_quarterly/C/x_test_c.csv')

In [102]:
x_train_c_combined = x_train_c.merge(y_train_c, left_on='date_forecast', right_on='time', how='left')
x_train_c_combined['observed'] = x_train_c_combined['calc_year'].isna().astype(int)
train_data_c = x_train_c_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis = 1)
x_test_c['observed'] = x_test_c['calc_year'].isna().astype(int)
test_data_c = x_test_c.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','date_forecast','location'], axis = 1)

# Trying with validation set equal to half of the estimated data


In [103]:
test_data_c['snow_density:kgm3_Q0'] = np.nan
test_data_c['snow_density:kgm3_Q1'] = np.nan
test_data_c['snow_density:kgm3_Q2'] = np.nan
test_data_c['snow_density:kgm3_Q3'] = np.nan
train_data_c.drop(columns= ['index'], inplace= True)

In [104]:

observed = train_data_c[train_data_c['observed'] == 1]
true =observed[observed['forecast_month'].isin([5, 6, 7])]
true

,date_forecast,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,...,forecast_year,forecast_month,forecast_day,forecast_hour,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3,pv_measurement,observed
4516,2020-05-01 00:00:00,3.5,3.5,3.5,3.5,1.260,1.257,1.255,1.253,NaN,...,2020,5,1,0,-0.000000,-0.000,-0.000000,-0.00000,0.0,1
4517,2020-05-01 01:00:00,3.5,3.5,3.5,3.5,1.250,1.250,1.250,1.249,4823.3,...,2020,5,1,1,-0.000000,-0.000,-0.000000,-0.00000,0.0,1
4518,2020-05-01 02:00:00,3.6,3.6,3.7,3.7,1.249,1.249,1.249,1.249,5061.4,...,2020,5,1,2,-0.000000,-0.000,-0.000000,-0.00000,0.0,1
4519,2020-05-01 03:00:00,3.8,3.8,3.9,3.9,1.249,1.249,1.248,1.248,4158.7,...,2020,5,1,3,-0.000000,0.000,0.000000,0.00000,0.0,1
4520,2020-05-01 04:00:00,4.0,4.1,4.1,4.2,1.248,1.247,1.247,1.246,3666.2,...,2020,5,1,4,13.497500,34.815,74.463295,124.23259,19.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14317,2021-07-31 19:00:00,8.6,8.7,8.7,8.7,1.209,1.209,1.209,1.210,1061.6,...,2021,7,31,19,21.331602,0.000,0.000000,0.00000,19.6,1
14318,2021-07-31 20:00:00,8.8,8.8,8.8,8.9,1.210,1.210,1.211,1.211,985.6,...,2021,7,31,20,0.000000,-0.000,-0.000000,-0.00000,0.0,1
14319,2021-07-31 21:00:00,8.9,9.0,9.0,9.1,1.212,1.212,1.213,1.213,942.7,...,2021,7,31,21,-0.000000,-0.000,-0.000000,-0.00000,0.0,1
14320,2021-07-31 22:00:00,9.1,9.1,9.1,9.1,1.214,1.214,1.215,1.215,824.9,...,2021,7,31,22,-0.000000,-0.000,-0.000000,-0.00000,0.0,1


In [105]:
from sklearn.model_selection import train_test_split

def split_dataset(train_data, date_column='forecast_month', observed_column='observed'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set includes approximately 50% of the estimated data, evenly distributed across months.
    Additionally, it includes about half of the observed data for May, June, and July, if present.
    The training set includes all months, excluding the observed data that is included in the validation set.
    
    :param train_data: The original training dataset as a pandas DataFrame.
    :param date_column: The name of the column that contains the month information.
    :param observed_column: The name of the column that indicates if the data is observed.
    :return: A tuple (training_set, validation_set)
    """
    # Work with a copy to avoid modifying the original DataFrame
    train_data = train_data.copy()
    train_data.sort_values(by='date_forecast', inplace=True)
    
    # Separate observed and estimated data
    estimated_data = train_data[train_data[observed_column] == 0]
    observed_data = train_data[train_data[observed_column] != 0]
    
    # Split the estimated data into training and validation sets
    estimated_train, estimated_val = train_test_split(
        estimated_data, test_size=0.5, random_state=42, stratify=estimated_data[date_column]
    )
    
    # Check if there are any observed data for May, June, and July
    if not observed_data[observed_data[date_column].isin([5, 6, 7])].empty:
        print("hellooo")
        observed_may_june_july = observed_data[observed_data[date_column].isin([5, 6, 7])]
        observed_train_mjj, observed_val_mjj = train_test_split(
            observed_may_june_july, test_size=0.5, random_state=42, stratify=observed_may_june_july[date_column]
        )
    else:
        
        observed_train_mjj = pd.DataFrame()
        observed_val_mjj = pd.DataFrame()
    
    # Combine the estimated and observed May, June, July data for the validation set
    validation_set = pd.concat([estimated_val, observed_val_mjj])
    validation_set.sort_values(by='date_forecast', inplace=True)
    
    # The rest of the observed data (excluding May, June, July) will be added to the training set
    observed_rest = observed_data[~observed_data[date_column].isin([5, 6, 7])]
    
    # Combine all training parts for the final training set
    training_set = pd.concat([estimated_train, observed_train_mjj, observed_rest])
    training_set.sort_values(by='date_forecast', inplace=True)
    
    return training_set, validation_set

# Create a copy of the DataFrame to pass to the function
train_data_copy = train_data_c.copy()
train_data_c, val_data_c = split_dataset(train_data_copy, 'forecast_month', 'observed')

print(len(train_data_c))
print(len(val_data_c))

hellooo
17987
3673


In [112]:
val_data_c

,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,ceiling_height_agl:m_Q1,...,forecast_year,forecast_month,forecast_day,forecast_hour,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3,pv_measurement,observed
4516,3.5,3.5,3.5,3.5,1.260,1.257,1.255,1.253,NaN,4823.3,...,2020,5,1,0,-0.0000,-0.00000,-0.000000,-0.00000,0.00,1
4517,3.5,3.5,3.5,3.5,1.250,1.250,1.250,1.249,4823.3,4882.8,...,2020,5,1,1,-0.0000,-0.00000,-0.000000,-0.00000,0.00,1
4520,4.0,4.1,4.1,4.2,1.248,1.247,1.247,1.246,3666.2,4135.0,...,2020,5,1,4,13.4975,34.81500,74.463295,124.23259,19.60,1
4523,4.5,4.4,4.4,4.3,1.245,1.244,1.243,1.241,2311.8,2388.3,...,2020,5,1,7,10.0616,5.34940,2.830300,825.82010,343.00,1
4524,4.2,4.2,4.1,4.0,1.240,1.238,1.236,1.234,2617.9,2549.7,...,2020,5,1,8,1826.2264,3005.36400,4354.760000,3552.79660,686.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21652,4.4,4.4,4.4,4.4,1.264,1.265,1.266,1.267,4222.6,3645.1,...,2023,4,30,16,2430.2446,2008.20860,1626.304900,1273.35270,125.44,0
21653,4.4,4.4,4.5,4.5,1.268,1.269,1.270,1.271,1912.7,1990.9,...,2023,4,30,17,969.2100,712.83844,501.360630,267.55440,190.12,0
21655,4.4,4.4,4.4,4.4,1.274,1.275,1.276,1.277,1474.2,1462.4,...,2023,4,30,19,0.0000,0.00000,-0.000000,-0.00000,50.96,0
21656,4.4,4.4,4.4,4.4,1.278,1.278,1.279,1.279,1427.3,1460.0,...,2023,4,30,20,-0.0000,-0.00000,-0.000000,-0.00000,2.94,0


In [106]:
train_data_c.drop(columns=['date_forecast'],inplace=True)
val_data_c.drop(columns=['date_forecast'],inplace=True)

In [107]:
predictor_c= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_c,verbosity = 2,presets='best_quality', time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231107_151221\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20


Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231107_151221\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   690.86 GB / 1022.87 GB (67.5%)
Train Data Rows:    17987
Train Data Columns: 189
Tuning Data Rows:    3673
Tuning Data Columns: 189
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (999.6, -0.0, 81.53573, 162.39102)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:      

[1000]	valid_set's l1: 19.049
[2000]	valid_set's l1: 18.653
[3000]	valid_set's l1: 18.4995
[4000]	valid_set's l1: 18.3863
[5000]	valid_set's l1: 18.3178
[6000]	valid_set's l1: 18.2882
[7000]	valid_set's l1: 18.2726
[8000]	valid_set's l1: 18.2561
[9000]	valid_set's l1: 18.2487
[10000]	valid_set's l1: 18.2425
[1000]	valid_set's l1: 19.2591
[2000]	valid_set's l1: 18.9407
[3000]	valid_set's l1: 18.7463
[4000]	valid_set's l1: 18.6531
[5000]	valid_set's l1: 18.6105
[6000]	valid_set's l1: 18.5893
[7000]	valid_set's l1: 18.5648
[8000]	valid_set's l1: 18.5545
[9000]	valid_set's l1: 18.5455
[10000]	valid_set's l1: 18.5395
[1000]	valid_set's l1: 18.9209
[2000]	valid_set's l1: 18.3359
[3000]	valid_set's l1: 18.1811
[4000]	valid_set's l1: 18.0963
[5000]	valid_set's l1: 18.0522
[6000]	valid_set's l1: 18.0323
[7000]	valid_set's l1: 18.0208
[8000]	valid_set's l1: 18.0037
[9000]	valid_set's l1: 17.9928
[10000]	valid_set's l1: 17.9827
[1000]	valid_set's l1: 18.2235
[2000]	valid_set's l1: 17.5311
[3000]	

	-31.5034	 = Validation score   (-mean_absolute_error)
	241.99s	 = Training   runtime
	3.85s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 735.47s of the 1235.27s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 19.6174
[2000]	valid_set's l1: 19.439
[3000]	valid_set's l1: 19.4027
[4000]	valid_set's l1: 19.3852
[5000]	valid_set's l1: 19.3751
[6000]	valid_set's l1: 19.3695
[7000]	valid_set's l1: 19.3669
[8000]	valid_set's l1: 19.3671
[9000]	valid_set's l1: 19.3669
[1000]	valid_set's l1: 19.7695
[2000]	valid_set's l1: 19.6355
[3000]	valid_set's l1: 19.6164
[4000]	valid_set's l1: 19.605
[5000]	valid_set's l1: 19.5986
[6000]	valid_set's l1: 19.5959
[7000]	valid_set's l1: 19.5934
[8000]	valid_set's l1: 19.593
[9000]	valid_set's l1: 19.5929
[10000]	valid_set's l1: 19.5927
[1000]	valid_set's l1: 19.4997
[2000]	valid_set's l1: 19.3857
[3000]	valid_set's l1: 19.3638
[4000]	valid_set's l1: 19.3627
[1000]	valid_set's l1: 19.084
[2000]	valid_set's l1: 18.8948
[3000]	valid_set's l1: 18.821
[4000]	valid_set's l1: 18.7885
[5000]	valid_set's l1: 18.7811
[6000]	valid_set's l1: 18.7783
[7000]	valid_set's l1: 18.778
[1000]	valid_set's l1: 20.1291
[2000]	valid_set's l1: 20.0469
[3000]	valid_

	-33.2733	 = Validation score   (-mean_absolute_error)
	271.53s	 = Training   runtime
	2.88s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 449.91s of the 949.7s of remaining time.
	-37.5343	 = Validation score   (-mean_absolute_error)
	49.4s	 = Training   runtime
	1.23s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 398.65s of the 898.45s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 2431.
	Ran out of time, early stopping on iteration 2688.
	Ran out of time, early stopping on iteration 2879.
	Ran out of time, early stopping on iteration 2953.
	Ran out of time, early stopping on iteration 3099.
	Ran out of time, early stopping on iteration 3225.
	Ran out of time, early stopping on iteration 3425.
	Ran out of time, early stopping on iteration 3996.
	-33.7268	 = Validation score   (-mean_absolute_error)
	38

[1000]	valid_set's l1: 17.5941


	-32.3731	 = Validation score   (-mean_absolute_error)
	19.08s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 481.7s of the 481.66s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-32.0868	 = Validation score   (-mean_absolute_error)
	13.72s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 467.45s of the 467.41s of remaining time.
	-31.9518	 = Validation score   (-mean_absolute_error)
	47.07s	 = Training   runtime
	1.17s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ... Training model for up to 418.63s of the 418.59s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-32.6924	 = Validation score   (-mean_absolute_error)
	62.59s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L2 ... Training model fo

In [108]:
predictor_c.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-31.162240,14.376106,1354.781825,0.000000,0.199031,3,True,18
1,NeuralNetFastAI_BAG_L2,-31.453003,10.905966,1114.579106,0.382136,161.063999,2,True,14
2,LightGBMXT_BAG_L1,-31.503435,3.853237,241.988032,3.853237,241.988032,1,True,3
3,WeightedEnsemble_L2,-31.503435,3.854236,242.158066,0.000999,0.170033,2,True,8
4,ExtraTreesMSE_BAG_L2,-31.757871,11.837406,963.221879,1.313576,9.706772,2,True,13
5,XGBoost_BAG_L2,-31.917246,10.672405,981.452165,0.148576,27.937057,2,True,15
6,RandomForestMSE_BAG_L2,-31.951825,11.689503,1000.580390,1.165673,47.065283,2,True,11
7,LightGBM_BAG_L2,-32.086834,10.625361,967.236444,0.101531,13.721337,2,True,10
8,LightGBMXT_BAG_L2,-32.373139,10.672569,972.599141,0.148739,19.084034,2,True,9
9,CatBoost_BAG_L2,-32.692435,10.605824,1016.108301,0.081994,62.593193,2,True,12


In [109]:
y_pred_c = predictor_c.predict(test_data_c)
y_pred_c=pd.DataFrame(y_pred_c,columns=['pv_measurement'])
y_pred_c #print the DataFrame

,pv_measurement
0,-0.542774
1,-0.688995
2,-0.896938
3,1.370289
4,14.306922
...,...
715,72.532654
716,43.460899
717,13.072128
718,1.831888


In [110]:
import os
output_dir = 'autogluon'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the dataframes into the folder
y_pred_c.to_csv(os.path.join(output_dir, 'y_pred_c.csv'))